In [10]:
# MacLight – GFLOPs & Time‑Budget Notebook  
# ------------------------------------------------------------
# This script can be executed as a Jupyter notebook (VS Code, JupyterLab)
# or run top‑to‑bottom as a plain Python file.  Cells are delimited by
# `# %%` markers so VS Code automatically recognises them.
#
# ▶ WHAT IT DOES
#   • loads MacLight networks (VAE, PolicyNet, ValueNet)
#   • measures MACs / GFLOPs with `ptflops`
#   • multiplies by call‑counts to give GFLOPs per EPISODE and for the
#     whole 80‑episode training run in the original paper
#   • prints a small timing estimate based on the *current GPU* 
#     (useful to compare laptop vs Colab A100)
# ------------------------------------------------------------

In [11]:
# %% 0  – Install dependencies if needed
%pip -q install ptflops 
%pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# %% 1  – Imports and repo path
import os, sys, time, math, torch
from pathlib import Path
from ptflops import get_model_complexity_info

#  ↳ if running in Colab, clone the repo automatically
if "google.colab" in sys.modules:
    if not Path("MacLight").exists():
        !git clone -q https://github.com/wissem18/MacLight.git
    repo_root = Path("MacLight")
    os.chdir(repo_root)
else:
    # local execution
    repo_root = Path.cwd()
    assert (repo_root / 'run_Ours.py').exists(), "Run inside the MacLight repo root"

sys.path.append(str(repo_root))

print(f"▶ Using repo at: {repo_root}\n")

▶ Using repo at: c:\Users\yousf\Bureau\PFE_important\Work\Codes\MacLight



In [17]:
# %% 2  – Load network definitions
from net.net import VAE, PolicyNet, ValueNet, Attention

# Dimensions taken from original config
STATE_DIM   = 33    
LATENT_DIM  = 10    
HIDDEN_DIM  = STATE_DIM * 2  
ACTION_DIM  = 8        

In [18]:
# %% 3  – Helper to compute GFLOPs

def gflops_and_params(model, inp_shape):
    macs, params = get_model_complexity_info(model, inp_shape,
                                             as_strings=False,
                                             print_per_layer_stat=False)
    return (macs * 2) / 1e9, params / 1e6


In [19]:
# %% 4  – Instantiate models
vae   = VAE(STATE_DIM, LATENT_DIM)
policy = PolicyNet(STATE_DIM, HIDDEN_DIM, ACTION_DIM)
value  = ValueNet(STATE_DIM, HIDDEN_DIM, LATENT_DIM)

In [20]:
# %% 5  – GFLOPs per forward pass
vae_g, vae_p = gflops_and_params(vae,   (STATE_DIM, 4, 4))
pol_g, pol_p = gflops_and_params(policy, (STATE_DIM ,))
val_g, val_p = gflops_and_params(value,  (STATE_DIM + LATENT_DIM,))

print(f"VAE                : {vae_g:.4f} GFLOPs | {vae_p:.2f} M params")
print(f"PolicyNet          : {pol_g:.4f} GFLOPs | {pol_p:.2f} M params")
print(f"ValueNet           : {val_g:.4f} GFLOPs | {val_p:.2f} M params")

VAE                : 0.0178 GFLOPs | 0.81 M params
PolicyNet          : 0.0000 GFLOPs | 0.01 M params
ValueNet           : 0.0000 GFLOPs | 0.01 M params


In [44]:
# %% 6  – Total GFLOPs per episode & full training run
T, K, E, L = 720, 16, 10, 80
vae_tot    = 3 * vae_g * T
actor_tot  = 3 * pol_g * T * K 
critic_tot = 3 * val_g * T * K * (1 + E)

per_episode = vae_tot + actor_tot + critic_tot
print(f"GFLOPs for VAE per episode  : {vae_tot: .2f}")
print(f"GFLOPs per episode   : {per_episode:.2f}")
print(f"GFLOPs for {L} eps : {per_episode*L:.2f} \n")


GFLOPs for VAE per episode  :  38.53
GFLOPs per episode   : 58.81
GFLOPs for 80 eps : 4704.73 



In [47]:
# Calculate GFLOPS for the method with single head attention
from net.net import Attention
GLOBAL_EMBED_DIM=32
attention=Attention(32,64,32)
policy = PolicyNet(STATE_DIM, HIDDEN_DIM, ACTION_DIM)
value  = ValueNet(STATE_DIM,(STATE_DIM+GLOBAL_EMBED_DIM)*2, GLOBAL_EMBED_DIM)
pol_g, pol_p = gflops_and_params(policy, (STATE_DIM ,))
val_g, val_p = gflops_and_params(value,  (STATE_DIM + GLOBAL_EMBED_DIM,))
att_g, att_p =gflops_and_params(attention,(16,STATE_DIM,))
print(f"Attention          : {att_g:.4f} GFLOPS | {att_p:.2f} M params")
print(f"PolicyNet          : {pol_g:.4f} GFLOPs | {pol_p:.2f} M params")
print(f"ValueNet           : {val_g:.4f} GFLOPs | {val_p:.2f} M params")

actor_att_tot  = 3 * pol_g * T * K * (1 + E)
critic_att_tot = 3 * val_g * T * K * (1 + E)
att_tot = 3 * att_g

per_episode = att_tot + actor_att_tot + critic_att_tot
print(f"GFLOPs per episode   : {per_episode:.2f}")


Attention          : 0.0004 GFLOPS | 0.01 M params
PolicyNet          : 0.0000 GFLOPs | 0.01 M params
ValueNet           : 0.0001 GFLOPs | 0.03 M params
GFLOPs per episode   : 25.35
